Importando Bibliotecas

In [1]:
import pyodbc
import pandas as pd
import warnings
import urllib
import os
from pyspark.sql import SparkSession
from sqlalchemy import create_engine


In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

Conexão com o banco

In [ ]:
senha = os.getenv('senha_bd_simples')

conn = pyodbc.connect(
    'DRIVER={SQL Server};' \
    'SERVER=den1.mssql7.gear.host;' \
    'DATABASE=vendasbicyclon;' \
    'UID=vendasbicyclon;' \
    f'PWD={senha};' \
    'MARS_Connection=yes;'
)

Função para executar Query's

In [ ]:
# Função para executar uma query
def executar_query(query):
    return pd.read_sql(query, conn)

lista_resposta = []
def exe_varias_queries(*args):
    for consulta in args:
        lista_resposta.append(pd.read_sql(consulta, conn))
    return lista_resposta

In [ ]:
# spark = SparkSession.builder.master('local[2]').appName('Teste').getOrCreate()
# spark = SparkSession.builder.getOrCreate()

In [ ]:
q1 = 'SELECT * FROM clientes'
q2 = 'SELECT * FROM produtos'

exe_varias_queries(q1, q2)

Criando conexão com SQLAlchemy

In [ ]:
params = urllib.parse.quote_plus(
    'DRIVER={SQL Server};' \
    'SERVER=den1.mssql7.gear.host;' \
    'DATABASE=vendasbicyclon;' \
    'UID=vendasbicyclon;' \
    f'PWD={senha};' \
    'MARS_Connection=yes;'
)

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

Produtos, total de pedidos e Qtd. vendidas

In [ ]:
produtos = '''
    SELECT  p.nomeProduto, 
    sum(ip.quantidade) as qtd_vendidos, 
    count(ip.idItem) as Total_Pedidos
    FROM produtos p
    LEFT JOIN itensPedido ip ON p.idProduto = ip.fkProduto
    GROUP BY p.nomeProduto
'''
executar_query(produtos)

Consolidado - View

In [ ]:
q_vw = '''
if OBJECT_ID('vwFatoVendas', 'V') IS NOT NULL
    DROP VIEW vwFatoVendas;

exec('
    CREATE VIEW vwFatoVendas AS
    SELECT  
        p.idPedido,
        p.dataPedido,
        c.nomeCliente,
        v.nomeVendedor,
        r.nomeRegiao,
        prod.nomeProduto,
        lp.nomeLinhaProduto,
        ip.prcoUnitario,
        (ip.quantidade * (ip.prcoUnitario - ip.desconto)) as ValorTotal,

    FROM pedidos p
    LEFT JOIN clientes c ON p.fkCliente = c.idCliente
    LEFT JOIN vendedores v ON p.fkVendedor = v.idVendedor
    LEFT JOIN regionais r ON v.fkRegional = r.idRegional
    LEFT JOIN itensPedido ip ON p.idPedido = ip.fkPedido
    LEFT JOIN produtos prod ON ip.fkProduto = prod.idProduto
    LEFT JOIN linhaProdutos lp ON p.fkLinhaProduto = lp.idLinhaProduto')
'''

cursor = conn.cursor()
cursor.execute(q_vw)
conn.commit()

df_fatoVendas = pd.read_sql('SELECT * FROM vwFatoVendas', conn)

df_fatoVendas.columns

In [2]:
print(os.getenv('senha_bd_simples'))

None
